## CNN Model using CIFAR-10 Dataset

CNN mainly work on image data and apply 'convolutional' filters, Have many layers with different task in each, we use max-pooling to reduce image size by reducing parameters.

In [26]:
# Basic steps in this cell
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 10
batch_size = 120
learning_rate = 3e-05

transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [27]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [28]:
# implement the model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 30, 5)
        self.conv3 = nn.Conv2d(30, 50, 5)
        self.fc1 = nn.Linear(50, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 50)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

# create an instance of the model
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer : 3 input channels, 6 output channels, 5 kernel size
        
        images = images.to(device) 
        labels = labels.to(device) 
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optim
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%2000==0:
            print(f'Epoch:{epoch+1}/{num_epochs}, step:{i+1}/{n_total_steps}, Loss:{loss.item():.4f}')
print('Finished Training')

Finished Training


In [29]:
# Testing on data

with torch.no_grad():
    n_correct=0
    n_samples=0
    n_class_correct= [0 for i in range(10)]
    n_class_samples= [0 for i in range(10)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        n_samples+=labels.size(0)
        n_correct+=(predicted==labels).sum().item()
        
        for i in range(min(batch_size, len(labels))):
            label = labels[i]
            pred = predicted[i]
            
            if (pred == label):
                n_class_correct[label]+=1
            n_class_samples[label]+=1
            
    acc = 100*n_correct/n_samples
    print(f'accuracy of network = {acc}%')

    for i in range(10):
        acc = 100*n_class_correct[i]/n_class_samples[i]
        print(f'accuracy of {classes[i]}:{acc}%')
        

accuracy of network = 9.96%
accuracy of plane:0.0%
accuracy of car:0.0%
accuracy of bird:0.0%
accuracy of cat:0.0%
accuracy of deer:0.0%
accuracy of dog:0.0%
accuracy of frog:0.0%
accuracy of horse:0.0%
accuracy of ship:0.0%
accuracy of truck:99.6%
